In [1]:
# import tensorflow as tf
import numpy as np
import h2o
import os
import pandas as pd
# from .feature_engineering_filter import Find_correct_port

In [2]:
train_data = pd.read_csv("../Datasets/ais_train.csv", delimiter="|")
test_data = pd.read_csv("../Datasets/ais_test.csv", delimiter=",")

In [3]:
train_data["time"] = pd.to_datetime(train_data["time"])
test_data["time"] = pd.to_datetime(test_data["time"])

In [4]:
train_data_preprocessed = train_data
train_data_preprocessed.loc[train_data_preprocessed["cog"] >= 360, "cog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["sog"] >= 1023, "sog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["rot"] == -128, "rot"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["heading"] == 511, "heading"] = np.nan


pattern = r"^\d{2}-\d{2} \d{2}:\d{2}$"
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].where(
    train_data_preprocessed["etaRaw"].str.match(pattern, na=False), np.nan
)


train_data_preprocessed=train_data_preprocessed.sort_values("time")

print(train_data_preprocessed.head())


train_data_preprocessed = train_data_preprocessed.groupby("vesselId").apply(lambda group: group.ffill().bfill()).reset_index(drop=True)


print(train_data_preprocessed.head())

train_data_preprocessed["heading"] = train_data_preprocessed["heading"].fillna(0)

train_data_preprocessed = train_data_preprocessed.dropna().reset_index(drop=True)




# Replace '00-' in etaRaw with the corresponding month and day from the 'time' column
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("00-", na=False),
    "01"
    + train_data_preprocessed["etaRaw"].str[2:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("-00", na=False),
    train_data_preprocessed["etaRaw"].str[:2]
    +"-01"
    + train_data_preprocessed["etaRaw"].str[5:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains(":60", na=False),
    train_data_preprocessed["etaRaw"].str[:9]+"59",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("60:", na=False),
    train_data_preprocessed["etaRaw"].str[:6]+"01:00",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("24:", na=False),
    train_data_preprocessed["etaRaw"].str[:6]+"23:59"
)


train_data_preprocessed["etaRaw"] = pd.to_datetime(
    train_data_preprocessed["time"].dt.year.astype(str)
    + "-"
    + train_data_preprocessed["etaRaw"]
    + ":00",
    format="%Y-%m-%d %H:%M:%S",
)


train_data_preprocessed["seconds_to_eta"] = (
    train_data_preprocessed["etaRaw"]-train_data_preprocessed["time"]
).dt.total_seconds()

train_data_preprocessed = train_data_preprocessed.drop(columns=["etaRaw"])

                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-01 00:00:25  284.0   0.7  0.0     88.0        0  01-09 23:00   
1 2024-01-01 00:00:36  109.6   0.0 -6.0    347.0        1  12-29 20:00   
2 2024-01-01 00:01:45  111.0  11.0  0.0    112.0        0  01-02 09:00   
3 2024-01-01 00:03:11   96.4   0.0  0.0    142.0        1  12-31 20:00   
4 2024-01-01 00:03:51  214.0  19.7  0.0    215.0        0  01-25 12:00   

   latitude  longitude                  vesselId                    portId  
0 -34.74370  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f  
1   8.89440  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689  
2  39.19065  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19  
3 -34.41189  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126  
4  35.88379   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3  


/var/tmp/ipykernel_25422/832757499.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_data_preprocessed = train_data_preprocessed.groupby("vesselId").apply(lambda group: group.ffill().bfill()).reset_index(drop=True)


                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-12 14:07:47  308.1  17.1 -6.0    316.0        0  01-08 06:00   
1 2024-01-12 14:31:00  307.6  17.3  5.0    313.0        0  01-14 23:30   
2 2024-01-12 14:57:23  306.8  16.9  5.0    312.0        0  01-14 23:30   
3 2024-01-12 15:18:48  307.9  16.9  6.0    313.0        0  01-14 23:30   
4 2024-01-12 15:39:47  307.0  16.3  7.0    313.0        0  01-14 23:30   

   latitude  longitude                  vesselId                    portId  
0   7.50361   77.58340  61e9f38eb937134a3c4bfd8b  61d376b393c6feb83e5eb50c  
1   7.57302   77.49505  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546  
2   7.65043   77.39404  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546  
3   7.71275   77.31394  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546  
4   7.77191   77.23585  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546  


In [5]:
train_latitude_radians = np.deg2rad(train_data_preprocessed["latitude"])
train_longitude_radians = np.deg2rad(train_data_preprocessed["longitude"])
train_cog_radians = np.deg2rad(train_data_preprocessed["cog"])
train_heading_radians = np.deg2rad(train_data_preprocessed["heading"])
train_hour = np.deg2rad(train_data_preprocessed["time"].dt.hour * 360 / 24)
train_day = np.deg2rad(train_data_preprocessed["time"].dt.day * 360 / 30)
train_month = np.deg2rad(train_data_preprocessed["time"].dt.month * 360 / 12)

print(train_data_preprocessed["latitude"].head(10))
print(train_latitude_radians.head(10))


train_latitude_sin = np.sin(train_latitude_radians)
train_latitude_cos = np.cos(train_latitude_radians)

train_longitude_sin = np.sin(train_longitude_radians)
train_longitude_cos = np.cos(train_longitude_radians)

train_cog_sin = np.sin(train_cog_radians)
train_cog_cos = np.cos(train_cog_radians)

train_heading_sin = np.sin(train_heading_radians)
train_heading_cos = np.cos(train_heading_radians)

train_hour_sin = np.sin(train_hour)
train_hour_cos = np.cos(train_hour)

train_day_sin = np.sin(train_day)
train_day_cos = np.cos(train_day)

train_month_sin = np.sin(train_month)
train_month_cos = np.cos(train_month)


train_data_preprocessed["latitude_sin"] = train_latitude_sin
train_data_preprocessed["latitude_cos"] = train_latitude_cos
train_data_preprocessed["longitude_sin"] = train_longitude_sin
train_data_preprocessed["longitude_cos"] = train_longitude_cos
train_data_preprocessed["cog_sin"] = train_cog_sin
train_data_preprocessed["cog_cos"] = train_cog_cos
train_data_preprocessed["heading_sin"] = train_heading_sin
train_data_preprocessed["heading_cos"] = train_heading_cos

train_data_preprocessed["hour_sin"] = train_hour_sin
train_data_preprocessed["hour_cos"] = train_hour_cos
train_data_preprocessed["day_sin"] = train_day_sin
train_data_preprocessed["day_cos"] = train_day_cos
train_data_preprocessed["month_sin"] = train_month_sin
train_data_preprocessed["month_cos"] = train_month_cos


train_data_preprocessed = train_data_preprocessed.drop(
    columns=["latitude", "longitude", "cog", "heading", "portId"], axis=1
)
print(train_data_preprocessed.head(10))

0    7.50361
1    7.57302
2    7.65043
3    7.71275
4    7.77191
5    7.81285
6    7.86929
7    7.92585
8    7.98258
9    8.03598
Name: latitude, dtype: float64
0    0.130963
1    0.132174
2    0.133525
3    0.134613
4    0.135645
5    0.136360
6    0.137345
7    0.138332
8    0.139322
9    0.140254
Name: latitude, dtype: float64
                 time   sog  rot  navstat                  vesselId  \
0 2024-01-12 14:07:47  17.1 -6.0        0  61e9f38eb937134a3c4bfd8b   
1 2024-01-12 14:31:00  17.3  5.0        0  61e9f38eb937134a3c4bfd8b   
2 2024-01-12 14:57:23  16.9  5.0        0  61e9f38eb937134a3c4bfd8b   
3 2024-01-12 15:18:48  16.9  6.0        0  61e9f38eb937134a3c4bfd8b   
4 2024-01-12 15:39:47  16.3  7.0        0  61e9f38eb937134a3c4bfd8b   
5 2024-01-12 15:54:48  16.1  5.0        0  61e9f38eb937134a3c4bfd8b   
6 2024-01-12 16:14:59  16.1 -6.0        0  61e9f38eb937134a3c4bfd8b   
7 2024-01-12 16:35:24  16.0  2.0        0  61e9f38eb937134a3c4bfd8b   
8 2024-01-12 16:55:24  16.0 -

In [6]:
def Last_known_location_training_data(data: pd.DataFrame, shift_lenghts) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data
        
    """
    all_test_data=pd.DataFrame()
    
    for shift_length in shift_lenghts:
    
        grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))

        grouped_data["time_diff"] = (
            grouped_data["time"].diff(-shift_length).dt.total_seconds().abs()
        )

        original_time_and_id = grouped_data[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ]

        shifted_data = grouped_data.shift(shift_length)
        shifted_data[
            [
                "last_latitude_sin",
                "last_latitude_cos",
                "last_longitude_sin",
                "last_longitude_cos",
            ]
        ] = shifted_data[["latitude_sin", "latitude_cos", "longitude_sin", "longitude_cos"]]

        shifted_data[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ] = original_time_and_id[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ]

        # Drops all values with nan values
        result = shifted_data.dropna().reset_index(drop=True)
        
        all_test_data = pd.concat([all_test_data, result], ignore_index=True)

    
    # Uncomment the line below if you want to remove the "time" column after processing
    # data = data.drop("time", axis=1)

    return all_test_data

In [7]:
print(train_data_preprocessed.columns)

train_data_shifted_df = Last_known_location_training_data(train_data_preprocessed,[1,5,10,50])

#train_data_shifted_df = train_data_shifted_df.drop(columns=["time"], axis=1)

Index(['time', 'sog', 'rot', 'navstat', 'vesselId', 'seconds_to_eta',
       'latitude_sin', 'latitude_cos', 'longitude_sin', 'longitude_cos',
       'cog_sin', 'cog_cos', 'heading_sin', 'heading_cos', 'hour_sin',
       'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos'],
      dtype='object')


/var/tmp/ipykernel_25422/1200682903.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))
/var/tmp/ipykernel_25422/1200682903.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))
/var/tmp/ipykernel_25422/1200682903.py:15: DeprecationWarning: DataFrameGr

In [8]:
def append_last_known_data_test(
    test_data: pd.DataFrame, known_data: pd.DataFrame
) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data
    """

    if not test_data["vesselId"].isin(known_data["vesselId"]).all():
        missing_vessels = test_data[
            ~test_data["vesselId"].isin(known_data["vesselId"])
        ]["vesselId"].unique()
        raise ValueError(
            f"The following vesselIds are missing in known_data: {missing_vessels}"
        )
    print(
        test_data[~test_data["vesselId"].isin(known_data["vesselId"])][
            "vesselId"
        ].unique()
    )

    grouped_data = (
        known_data.sort_values("time")
        .groupby("vesselId")
        .tail(1)
        .reset_index(drop=True)
    )
    original_time = test_data[["time"]]
    test_data = test_data.drop("time", axis=1)

    result = pd.merge(test_data, grouped_data, how="left", on="vesselId")

    result["time_diff"] = (original_time["time"] - result["time"]).dt.total_seconds()

    return result

In [9]:
test_data_with_last_known_df = append_last_known_data_test(
    test_data, train_data_preprocessed
)
test_data_with_last_known_df[
    [
        "last_latitude_sin",
        "last_latitude_cos",
        "last_longitude_sin",
        "last_longitude_cos",
    ]
] = test_data_with_last_known_df[
    [
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ]
]
test_data_with_last_known_df = test_data_with_last_known_df.drop(
    columns=[
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ],
    axis=1,
)

print(test_data_with_last_known_df.head())

[]
   ID                  vesselId  scaling_factor                time   sog  \
0   0  61e9f3aeb937134a3c4bfe3d             0.3 2024-05-07 23:48:16   0.0   
1   1  61e9f473b937134a3c4c02df             0.3 2024-05-07 23:57:16   0.0   
2   2  61e9f469b937134a3c4c029b             0.3 2024-05-07 23:59:08  18.7   
3   3  61e9f45bb937134a3c4c0221             0.3 2024-05-07 23:52:34   0.1   
4   4  61e9f38eb937134a3c4bfd8d             0.3 2024-05-07 23:51:29   0.3   

   rot  navstat  seconds_to_eta   cog_sin   cog_cos  ...  hour_cos   day_sin  \
0  0.0        5       -133396.0  0.006981 -0.999976  ...  0.965926  0.994522   
1  0.0        5       -521836.0  0.417867  0.908508  ...  0.965926  0.994522   
2  0.0        0         45952.0  0.139173  0.990268  ...  0.965926  0.994522   
3  0.0        1        -81454.0 -0.625243  0.780430  ...  0.965926  0.994522   
4  0.0        2        101311.0 -0.933580  0.358368  ...  0.965926  0.994522   

    day_cos  month_sin  month_cos  time_diff  last_la

In [10]:
h2o.init(max_mem_size="24g")

train_data_shifted = h2o.H2OFrame(train_data_shifted_df)
test_data_with_last_known = h2o.H2OFrame(test_data_with_last_known_df)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Debian-2deb11u1); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Debian-2deb11u1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/tmp/tmpkpvilywk
  JVM stdout: /var/tmp/tmpkpvilywk/h2o_jupyter_started_from_python.out
  JVM stderr: /var/tmp/tmpkpvilywk/h2o_jupyter_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 22 days
H2O_cluster_name:,H2O_from_python_jupyter_0l2a4b
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,24 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
train_data_shifted_without_validation, validation_data_shifted = (
    train_data_shifted.split_frame(ratios=[0.8], seed=42)
)

In [12]:
features_lat = [
    #"vesselId",
    "cog_sin",
    "cog_cos",
    "sog",
    "rot",
    "heading_sin",
    "heading_cos",
    "navstat",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    "hour_sin",
    "hour_cos",
    "day_sin",
    "day_cos",
    "month_sin",
    "month_cos",
]
features_long = [
    #"vesselId",
    "cog_sin",
    "cog_cos",
    "sog",
    "rot",
    "heading_sin",
    "heading_cos",
    "navstat",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    "latitude_sin",
    "latitude_cos",
    "hour_sin",
    "hour_cos",
    "day_sin",
    "day_cos",
    "month_sin",
    "month_cos",
]
target_long_sin = "longitude_sin"
target_long_cos = "longitude_cos"
target_lat_sin = "latitude_sin"
target_lat_cos = "latitude_cos"

In [13]:
params = {
    "ntrees": 300,  # Maximum number of trees
    "max_depth": 12,  # Maximum depth of each tree
    "min_rows": 15,  # Minimum number of rows per leaf
    "learn_rate": 0.1,  # Learning rate
    "sample_rate": 0.8,  # Row sample rate per tree
    "col_sample_rate": 0.7,  # Column sample rate per tree
    "reg_lambda": 1.0,  # L2 regularization term
    "reg_alpha": 0.1,  # L1 regularization term
    "seed": 42,  # Random seed for reproducibility
    "stopping_rounds": 10,  # Early stopping rounds
    "stopping_metric": "AUTO",  # Metric for early stopping
    "stopping_tolerance": 0.001  # Tolerance for early stopping
}
#score_eval_metric_only if you want to predict only on the evaluation dataset, could help against overfitting

gbm_lat_sin = h2o.estimators.H2OXGBoostEstimator(
 **params
)
gbm_lat_cos = h2o.estimators.H2OXGBoostEstimator(
**params
)
gbm_long_sin = h2o.estimators.H2OXGBoostEstimator(
**params
)
gbm_long_cos = h2o.estimators.H2OXGBoostEstimator(
**params
)

# gbm_cog = h2o.estimators.H2OXGBoostEstimator()
# gbm_sog = h2o.estimators.H2OXGBoostEstimator()
# gbm_rot = h2o.estimators.H2OXGBoostEstimator()
# gbm_heading = h2o.estimators.H2OXGBoostEstimator()
# gbm_navstat = h2o.estimators.H2OXGBoostEstimator()
# # gbm_etaRaw = h2o.esti#mators.H2OXGBoostEstimator() #Remove etaRaw because it requires preprocessing
# # gbm_portId = h2o.estimators.H2OXGBoostEstimator()

In [ ]:
gbm_lat_sin.train(
    x=features_lat,
    y=target_lat_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame= validation_data_shifted
)

xgboost Model Build progress: |

/opt/conda/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: early stopping is enabled but neither score_tree_interval or score_each_iteration are defined. Early stopping will not be reproducible!
  warnings.warn(mesg["message"], RuntimeWarning)


████████

In [ ]:
features_lat.append("latitude_sin")
gbm_lat_cos.train(
    x=features_lat,#.append(latitude_sin)
    y=target_lat_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame= validation_data_shifted

)

In [ ]:
performance_lat_sin = gbm_lat_sin.model_performance(test_data=validation_data_shifted)
performance_lat_cos = gbm_lat_cos.model_performance(test_data=validation_data_shifted)


# Print the performance metrics
print(performance_lat_sin)
print(performance_lat_cos)

In [ ]:
gbm_long_sin.train(
    x=features_long,
    y=target_long_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame= validation_data_shifted

)

In [ ]:
features_long.append("longitude_sin")
gbm_long_cos.train(
    x=features_long,#.append("longitude_sin")
    y=target_long_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame= validation_data_shifted

)

In [ ]:
performance_long_sin = gbm_long_sin.model_performance(test_data=validation_data_shifted)
performance_long_cos = gbm_long_cos.model_performance(test_data=validation_data_shifted)

print(performance_long_sin)
print(performance_long_cos)

In [ ]:
test_data_with_predicted_lat = test_data_with_last_known

lat_predictions_sin = gbm_lat_sin.predict(test_data_with_last_known)
test_data_with_last_known["latitude_sin"]=lat_predictions_sin
lat_predictions_cos = gbm_lat_cos.predict(test_data_with_last_known)
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

test_data_with_predicted_lat["latitude_sin"] = lat_predictions_sin
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

long_predictions_sin = gbm_long_sin.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_sin"]=long_predictions_sin
long_predictions_cos = gbm_long_cos.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_cos"]=long_predictions_cos


In [ ]:
# Convert sine and cosine values back to radians
lat_predictions_sin = lat_predictions_sin.as_data_frame()
lat_predictions_cos = lat_predictions_cos.as_data_frame()
long_predictions_sin = long_predictions_sin.as_data_frame()
long_predictions_cos = long_predictions_cos.as_data_frame()


lat_predictions_radians = np.arctan2(lat_predictions_sin, lat_predictions_cos)
long_predictions_radians = np.arctan2(long_predictions_sin, long_predictions_cos)

# Convert radians to degrees
lat_predictions_degrees = np.rad2deg(lat_predictions_radians)
long_predictions_degrees = np.rad2deg(long_predictions_radians)

# Print the first few rows to verify the conversion
print(lat_predictions_degrees.head())
print(long_predictions_degrees.head())

In [ ]:
def create_prediction_visualization_data(validation_data):
    lat_val_sin = gbm_lat_sin.predict(validation_data)
    lat_val_cos = gbm_lat_cos.predict(validation_data)
    long_val_sin = gbm_long_sin.predict(validation_data)
    long_val_cos = gbm_long_cos.predict(validation_data)

    lat_val_sin = lat_val_sin.as_data_frame()
    lat_val_cos = lat_val_cos.as_data_frame()
    long_val_sin = long_val_sin.as_data_frame()
    long_val_cos = long_val_cos.as_data_frame()

    validation_data=validation_data.as_data_frame()

    lat_val_radians = np.arctan2(lat_val_sin, lat_val_cos)
    long_val_radians = np.arctan2(long_val_sin, long_val_cos)

    evaluation_lat_radians=np.arctan2(validation_data["latitude_sin"], validation_data["latitude_cos"])
    evaluation_long_radians=np.arctan2(validation_data["longitude_sin"], validation_data["longitude_cos"])


    # Convert radians to degrees
    lat_val_degrees = np.rad2deg(lat_val_radians)
    long_val_degrees = np.rad2deg(long_val_radians)

    evaluation_lat_degrees=np.rad2deg(evaluation_lat_radians)
    evaluation_long_degrees=np.rad2deg(evaluation_long_radians)


    eval_predictions = pd.concat([lat_val_degrees, long_val_degrees], axis=1)

    eval_actual= pd.concat([evaluation_lat_degrees,evaluation_long_degrees], axis=1)

    eval_predictions.columns = ["latitude_predicted", "longitude_predicted"]
    eval_actual.columns = ["latitude", "longitude"]

    
    eval = pd.DataFrame()
    eval[["latitude_predicted","longitude_predicted"]] = eval_predictions
    eval[["latitude", "longitude"]] = eval_actual
    eval[["vesselId","time"]] =validation_data[["vesselId","time"]]
    eval.to_csv("eval_predictions.csv")

#create_prediction_visualization_data(validation_data_shifted)

In [ ]:
predictions = pd.concat([lat_predictions_degrees, long_predictions_degrees], axis=1)
predictions.columns = ["latitude_predicted", "longitude_predicted"]

In [ ]:
predictions["ID"] = test_data["ID"]
predictions = predictions[["ID", "longitude_predicted", "latitude_predicted"]]

In [ ]:
print(predictions.columns)

In [ ]:
predictions.to_csv("predictions.csv", index=False)